**Project 2**  
*AEEM 2058*  
*Date*:11/30/2025  
Hayden Mays, Logan Mueller, Evan Wills  
___

In [1]:
# Haydens Version
# Imports
import numpy as np
import time
import matplotlib.pyplot as plt

---

In [2]:
def RK4(y,system:callable,h,t):
    k1=system(t,y)
    k2=system(t+h/2,y+(k1/2)*h)
    k3=system(t+h/2,y+(k2/2)*h)
    k4=system(t+h,y+(k3)*h)  
        
        # Update y value using the weighted average of the slopes
    y=y+(1/6)*(k1+2*k2+2*k3+k4)*h
    return y

def Euler(t_0:float,t_f:float,h:float,S_initial:float,dSdt:callable)-> tuple [np.ndarray:'S values',np.ndarray]:
    ''' Runs Euler method
    
    Args:
        t_0: the initial time
        t_f: the finial time
        h: the step length between the initial and finial time
        S_initial: the initial condition
        dSdt: the ODE for S
        
    Returns:
        tuple of 2 arrays
        (t_span,S)
        
    '''
    t_span=np.arange(t_0,t_f+h,h) # creats span
    
    S=np.zeros((len(t_span),4)) # Creates array for outputs
    S[0]=S_initial #Sets intial value
    
    # Runs euler aproximation
    for i in range(len(t_span)-1):
        S[i+1]=S[i]+h*dSdt(t_span[i],S[i])
    
    # Returns t_span and the outputs
    return t_span,S

def system(t,state):
    x,z,v_x,v_z=state
    g=9.8 #m/s^2
    c=10e-3 #m^-1
    dx=v_x
    dz=v_z
    v=np.sqrt(v_x**2+v_z**2)
    dv_x=-c*v_x*v
    dv_z=-g-c*v_z*v
    return np.array([dx,dz,dv_x,dv_z])

In [10]:
# A)
t_initial=0
t_finial=3
t_step=0.0001
N=int((t_finial-t_initial)/t_step)



v0=1000 #m/s
theta0=10

y0=np.array([0,0,v0*np.cos(np.deg2rad(theta0)),v0*np.sin(np.deg2rad(theta0))])

t_c,v=Euler(t_initial,t_finial,t_step,y0,system)
v_x=v.T[2]
trapx=(t_step/2)*(v_x[0]+ 2*sum(v_x[1:N-1])+v_x[N-1])

v_z=v.T[3]
trapz=(t_step/2)*(v_z[0]+ 2*sum(v_z[1:N-1])+v_z[N-1])

print(f'Initial Velocity: {v0:0.03f} m/s')
print(f'Initial Angle: {theta0:0.03f}°\n')

print('Time'.center(10)+'|'+'X Velocity'.center(15)+'|'+'Y Velocity'.center(14))
print(''.center(42,'-'))
for t,vx,vz in zip(t_c,v_x,v_z):
    print(f'{t:7.03f} s | {vx:8.03f} m/s | {vz:8.03f} m/s'.center(42),end='\r',flush=True)
    #time.sleep(t_step)

print(f'\n\nRange: {trapx:0.03f} m | {trapx/1609:0.03f} Mile(s)')

print(f'Finial Z: {trapz:0.03f} m\n')
print(v.T[0])
print(v.T[1])

Initial Velocity: 1000.000 m/s
Initial Angle: 10.000°

   Time   |   X Velocity  |  Y Velocity  
------------------------------------------
   3.000 s |   31.932 m/s |   -9.529 m/s  

Range: 338.878 m | 0.211 Mile(s)
Finial Z: 36.355 m

[0.00000000e+00 9.84807753e-02 1.96863070e-01 ... 3.38922596e+02
 3.38925790e+02 3.38928983e+02]
[0.00000000e+00 1.73648178e-02 3.47121727e-02 ... 3.63652657e+01
 3.63643129e+01 3.63633600e+01]


This is what I got so far, this is forward solving it, while the question needs us to back solve for velocity and angle

In [9]:

def z_zero(x0,z0,theta,v,t_initial,dx,t_step):
    vx=v*np.cos(np.deg2rad(theta))
    vz=v*np.sin(np.deg2rad(theta))
    sol=np.array([x0,z0,vx,vz])
    t=t_initial
    z=1
    while z>0.0:
        sol=RK4(sol,system,t_step,t)
        t+=t_step
        z=sol[1]
    if sol[0]<dx:
      return None
    return t



def find(x0,z0,theta,v_range,dx,t_initial,t_step):
    for v in v_range:    
        t_f=z_zero(x0,z0,theta,v,t_initial,dx,t_step)
        if t_f!=None:
            return [theta,v,t_f]
    return None



def solve(target,t_initial,t_step,theta,v_min,v_max,v_step=1000.0,min_step=10e-3):
    
    v_range=np.arange(v_min,v_max+v_step,v_step)
    sol=find(0,0,theta,v_range,target,t_initial,t_step)
    if sol==None:
        print('Increase Speed Range')
        return None
    if v_step>min_step:
        return solve(target,t_initial,t_step,theta,sol[1]-v_step,sol[1],v_step/10)
    else:
         return sol
    
sol=solve(804.5,0,0.001,15,0,50000)

print(sol)

[15, np.float64(47578.180000000015), 10.330999999999714]


In [ ]:
# Validation


---

In [ ]:
# B)

Discusion

---

In [ ]:
# C) 😦